#Use isto se for a primeira vez usando o datalake da basedosdados.org: <br>
   pip install basedosdados
<br><br>
#Use para baixar a base como microdados_violencia.csv: <br>
   import basedosdados as bd <br>
   df = bd.read_table(dataset_id='br_ms_sinan', table_id='microdados_violencia', billing_project_id="violenciasinan", use_bqstorage_api = True) <br>
   df.to_csv(path_or_buf = 'microdados_violencia.csv')

In [18]:
# importar bibliotecas usadas
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing

In [9]:
print(tf.__version__)
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(physical_devices)
except:
    print("No GPU")

2.9.1
[]


In [6]:
# carregar dados em um dataframe pandas
# para usar o arquivo .csv baixe da pasta do sharepoint ou com os comandos disponíveis no topo do notebook
df = pd.read_csv("C:/Users/rozan/OneDrive - Fundacao Getulio Vargas - FGV/Eletivas/AdvLearning/microdados_violencia.csv")

In [7]:
# resumo dos dados
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df)

,Unnamed: 0,ano,data_notificacao,id_categoria_cid10,id_subcategoria_cid10,tipo_notificacao,id_uf_notificacao,id_municipio_notificacao,id_municipio_6_notificacao,id_unidade_notificacao,id_regional_saude_notificacao,data_ocorrencia,id_uf_ocorrencia,id_municipio_ocorrencia,id_municipio_6_ocorrencia,hora_ocorrencia,local_ocorrencia,outro_local_ocorrencia,outras_vezes_ocorrencia,id_uf_residencia,id_municipio_residencia,id_municipio_6_residencia,id_regional_saude_residencia,idade_paciente,sexo_paciente,gestante_paciente,raca_paciente,escolaridade_paciente,ocupacao_paciente,estado_civil_paciente,orientacao_sexual_paciente,identidade_genero_paciente,motivacao_violencia,violencia_relacionada_trabalho,emitiu_cat,deficiencia_transtorno_paciente,deficiencia_fisica_paciente,deficiencia_mental_paciente,deficiencia_visual_paciente,deficiencia_auditiva_paciente,transtorno_mental_paciente,transtorno_comportamental_paciente,outras_deficiencias_paciente,quais_outras_deficiencias_paciente,lesao_autoprovocada,ocorreu_violencia_fisica,ocorreu_violencia_psicologica,ocorreu_tortura,ocorreu_violencia_sexual,ocorreu_trafico_ser_humano,ocorreu_violencia_financeira,ocorreu_negligencia_abandono,ocorreu_trabalho_infantil,ocorreu_intervencao_legal,ocorreu_outra_violencia,ocorreu_qual_outra,meio_forca,meio_enforcamento,meio_objeto_contundente,meio_objeto_perfurante,meio_objeto_quente,meio_envenenamento,meio_arma_fogo,meio_ameaca,meio_outros,meio_qual_outro,houve_assedio,houve_estupro,houve_pornografia_infantil,houve_exploracao_sexual,houve_outra_violencia_sexual,houve_qual_outra_violencia_sexual,profilaxia_dst,profilaxia_hiv,profilaxia_hepatite_b,coleta_sangue,coleta_semen,coleta_secrecao_vaginal,profilaxia_contraceptivo,aborto,numero_envolvidos_violencia,autor_pai,autor_mae,autor_padrasto,autor_madrasta,autor_conjugue,autor_ex_conjugue,autor_namorado_a,autor_ex_namorado_a,autor_filho_a,autor_desconhecido,autor_irmao,autor_conhecido,autor_cuidador,autor_patrao_chefe,autor_institucional,autor_policial,autor_propria_pessoa,autor_outros,autor_relacao_outros,autor_sexo,autor_usou_alcool,encaminhamento_saude,encaminhamento_assistencia_social,encaminhamento_educacao,encaminhamento_atendimento_mulher,encaminhamento_conselho_tutelar,encaminhamento_conselho_idoso,encaminhamento_delegacia_idoso,encaminhamento_direitos_humanos,encaminhamento_mpu,encaminhamento_delegacia_crianca,encaminhamento_delegacia_mulher,encaminhamento_delegacia,encaminhamento_justica_infancia_juventude,encaminhamento_defensoria_publica,data_encerramento
0,0,2009,2009-11-03,Y09,Y00.4,2,12,1200104.0,120010,2001500.0,1937.0,2009-11-03,12.0,1200104.0,120010,02:30:00,6.0,NaN,0.0,12,1200104.0,120010,1937.0,26.0,1.0,6.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,0.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.0,8.0,8.0,8.0,8.0,NaN,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2009-11-04
1,1,2009,2009-02-04,Y09,NaN,2,27,2700300.0,270030,3015408.0,1537.0,2009-02-04,27.0,2700300.0,270030,17:58:00,NaN,NaN,NaN,27,2700300.0,270030,1537.0,27.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2009-02-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [15]:
# separando labels e features, features transformadas em array
features = df.copy()
labels = features.pop('outras_vezes_ocorrencia')
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Do a calculation using is
result = 2*input + 1

# the result doesn't have a value
result
calc = tf.keras.Model(inputs=input, outputs=result)

In [16]:
print(calc(1).numpy())
print(calc(2).numpy())

3.0
5.0


In [17]:
inputs = {}

for name, column in features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'Unnamed: 0': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Unnamed: 0')>,
 'ano': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'ano')>,
 'data_notificacao': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'data_notificacao')>,
 'id_categoria_cid10': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'id_categoria_cid10')>,
 'id_subcategoria_cid10': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'id_subcategoria_cid10')>,
 'tipo_notificacao': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tipo_notificacao')>,
 'id_uf_notificacao': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'id_uf_notificacao')>,
 'id_municipio_notificacao': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'id_municipio_notificacao')>,
 'id_municipio_6_notificacao': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'id_municipio_6_notificacao')>,
 'id_unidade_notificacao': <KerasTensor: sha

In [19]:
# concatenar e normalizar
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(df[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 103) dtype=float32 (created by layer 'normalization')>

In [11]:
# modelo de regressão
# SGD como otimizador
modelo = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

modelo.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.SGD())

In [14]:
modelo.fit(features, labels, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).